In [1]:
from EM_part import *

In [86]:
args_result = np.load('./numpy/arg_all_result_pycharm.npy',allow_pickle=True).tolist()

In [88]:
args_result

[[0.3,
  array([[-1.42777011],
         [-0.69358169],
         [-1.7471022 ],
         [-0.20915441],
         [-2.91410903],
         [ 0.13284397]]),
  [array([[ 0.96244846],
          [-0.48296374],
          [ 6.49305956]]),
   array([[ 0.16319068, -0.01633969, -0.78765068],
          [-0.01633969,  0.07647646, -0.10008035],
          [-0.78765068, -0.10008035,  6.41657906]]),
   0.05172389523847257],
  [array([[ 0.91623615],
          [-0.3941451 ],
          [ 5.84141011]]),
   array([[ 0.13744287,  0.0289401 , -0.75884416],
          [ 0.0289401 ,  0.16812421, -0.33487241],
          [-0.75884416, -0.33487241,  5.49046597]]),
   0.05750825486541421],
  [array([[ 0.71595196],
          [-0.51565889],
          [ 7.68896639]]),
   array([[ 0.10387953, -0.06726315, -0.29754407],
          [-0.06726315,  0.43814228, -0.91442471],
          [-0.29754407, -0.91442471,  6.5213332 ]]),
   0.036274010864397084],
  [array([[-0.56294419],
          [ 0.50192595],
          [-8.74370562]])

In [87]:
for i in range(100):
    print(get_hat_rho_l(test_dataset[i],*args_result))

[[0.]
 [1.]]
[[1.]
 [0.]]
[[0.]
 [1.]]
[[1.]
 [0.]]
[[0.]
 [1.]]
[[1.]
 [0.]]
[[1.]
 [0.]]
[[1.]
 [0.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[1.]
 [0.]]
[[1.]
 [0.]]
[[1.]
 [0.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[1.]
 [0.]]
[[0.]
 [1.]]
[[1.]
 [0.]]
[[0.]
 [1.]]
[[1.]
 [0.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[1.]
 [0.]]
[[0.]
 [1.]]
[[1.]
 [0.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[1.]
 [0.]]
[[0.]
 [1.]]
[[1.]
 [0.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[1.]
 [0.]]
[[1.]
 [0.]]
[[0.]
 [1.]]
[[1.]
 [0.]]
[[1.]
 [0.]]
[[0.]
 [1.]]
[[1.]
 [0.]]
[[0.]
 [1.]]
[[1.]
 [0.]]
[[1.]
 [0.]]
[[0.]
 [1.]]
[[1.]
 [0.]]
[[1.]
 [0.]]
[[1.]
 [0.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[1.]
 [0.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[0.]
 [1.]]
[[1.]
 [0.]]
[[0.]
 [1.]]

In [21]:
def mode_para_selection(instance_q,dk_list,*args):
    """
    *args [pi,w,[mu,sigma],[mu,sigma]],[pi,w,[mu,sigma],[mu,sigma]]
    return [pi,w,mu,...]
    """
    mode_q = np.argmax(get_hat_rho_l(instance_q,*args))
    selected_args = []
    args_q = args[mode_q]
    selected_args.append(dk_list[mode_q]) # dk
    selected_args.append(args_q[1]) # wk
    #selected_args.appned(args_q[1])
    for idx in range(3):
        selected_args.append(args_q[-1][idx])
    return selected_args

select_args = mode_para_selection(test_dataset[0],[-1,1], *args_result)   

In [20]:
test_dataset[i]["Phi_l"]

array([[1.00000e+00, 2.00000e-03, 4.00000e-06],
       [1.00000e+00, 4.00000e-03, 1.60000e-05],
       [1.00000e+00, 6.00000e-03, 3.60000e-05],
       [1.00000e+00, 8.00000e-03, 6.40000e-05],
       [1.00000e+00, 1.00000e-02, 1.00000e-04],
       [1.00000e+00, 1.20000e-02, 1.44000e-04],
       [1.00000e+00, 1.40000e-02, 1.96000e-04],
       [1.00000e+00, 1.60000e-02, 2.56000e-04],
       [1.00000e+00, 1.80000e-02, 3.24000e-04],
       [1.00000e+00, 2.00000e-02, 4.00000e-04],
       [1.00000e+00, 2.20000e-02, 4.84000e-04],
       [1.00000e+00, 2.40000e-02, 5.76000e-04],
       [1.00000e+00, 2.60000e-02, 6.76000e-04],
       [1.00000e+00, 2.80000e-02, 7.84000e-04],
       [1.00000e+00, 3.00000e-02, 9.00000e-04],
       [1.00000e+00, 3.20000e-02, 1.02400e-03],
       [1.00000e+00, 3.40000e-02, 1.15600e-03],
       [1.00000e+00, 3.60000e-02, 1.29600e-03],
       [1.00000e+00, 3.80000e-02, 1.44400e-03],
       [1.00000e+00, 4.00000e-02, 1.60000e-03],
       [1.00000e+00, 4.20000e-02, 1.7640

In [23]:
def get_post_Sigma_q_and_mu_q(instance_q,*args):
    """
    args:select_args,[pi,w,mu,...]
    """
    [hat_D,w,mu_Gamma,C_Gamma,sigma_2] = args
    x_q = instance_q["input"]
    Phi_q = instance_q["Phi_l"]
    Sigma_q = np.linalg.inv((Phi_q.T @ Phi_q) / sigma_2 + np.linalg.inv(C_Gamma))
    mu_q = Sigma_q @ ((Phi_q.T @ x_q @ w) / sigma_2 + np.linalg.inv(C_Gamma) @ mu_Gamma)
    return mu_q, Sigma_q
mu_q,Sigma_q = get_post_Sigma_q_and_mu_q(test_dataset[0],*select_args)

In [27]:
import scipy

In [75]:
def get_CDF_t(instance_q,t,*args):
    n_q = len(instance_q["input"])
    Phi_q = instance_q["Phi_l"]
    [hat_D,w,mu_Gamma,C_Gamma,sigma_2] = args
    t_nl_t = n_q + t
    Phi_t_nl_t = np.array([1, t_nl_t / 500, (t_nl_t / 500) * (t_nl_t / 500)], 
                          dtype=np.float64).reshape(1,3)
    mu_q, Sigma_q = get_post_Sigma_q_and_mu_q(instance_q,*args)
    mu_t_nl_t_0 = Phi_q[-1] @ mu_q
    mu_t_nl_t = Phi_t_nl_t @ mu_q
    sigma_t_nl_t = Phi_t_nl_t @ Sigma_q @ Phi_t_nl_t.T
    if mu_t_nl_t_0<hat_D:
        A = ((hat_D - mu_t_nl_t) / (np.sqrt(sigma_t_nl_t))).item()
    else :
        A = ((-hat_D + mu_t_nl_t) / (np.sqrt(sigma_t_nl_t))).item()
    Phi_t = 1 - scipy.stats.norm.cdf(A)
    return Phi_t

get_CDF_t(test_dataset[0],69, *select_args)    
    

0.9985843023033045

In [79]:
def get_single_RUL(instance_q,dk_list,*args):
    """
    *args:[pi,w,[mu,sigma],[mu,sigma]],[pi,w,[mu,sigma],[mu,sigma]] previous results
    """
    select_args = mode_para_selection(instance_q,dk_list, *args)
    box = []
    for t in range(500):
        A = get_CDF_t(instance_q,t,*select_args)
        B = get_CDF_t(instance_q,0,*select_args)
        C = (A-B)/(1-B)
        D = abs(C-0.5)
        box.append(D)
    RUL_q = box.index(min(box)) + 1
    return RUL_q

get_single_RUL(test_dataset[1],[-1,1],*args_result)
    

54

In [84]:
def get_RUL(instance, dk_list, *args):
    length = len(instance)
    RUL = []
    for idx in tqdm(range(length)):
        RUL_q = get_single_RUL(instance[idx], dk_list, *args)
        RUL.append(RUL_q)
    return np.array(RUL)

In [81]:
get_RUL(test_dataset,[-1,1],*args_result)

array([ 70,  54,  82, 109, 500, 120,  56,  57,  56,  85,  76,  92, 107,
        39, 226,  93, 435, 500, 212, 132,  22, 284, 221,   6,  51,   9,
       142, 248, 108, 193, 283,  51,  82,  69, 163, 152, 224,  27,   1,
        84, 240, 113, 317,  60, 263,  13, 123,  77, 498,  16,  87, 156,
        66, 142,  57, 113, 112,  43,  49, 116, 265, 181, 302,  74, 226,
       106, 209, 500, 287,  81,  74,  61, 209, 337,  74,  19, 153,  40,
        22, 500,  38,  12, 170,  84, 103,  36, 145, 151,  98,  54, 116,
        23, 160,   5, 301, 381, 238,  25,   5,  88])

In [83]:
#RUL
path = DATA_PATH+'RUL.csv'
RUL_frame = pd.read_csv(path, header=None)
RUL = RUL_frame.values[:, 0]
RUL

array([ 44,  51,  27, 120, 101,  99,  71,  55,  55,  66,  77, 115, 115,
        31, 108,  56, 136, 132,  85,  56,  18, 119,  78,   9,  58,  11,
        88, 144, 124,  89,  79,  55,  71,  65,  87, 137, 145,  22,   8,
        41, 131, 115, 128,  69, 111,   7, 137,  55, 135,  11,  78, 120,
        87,  87,  55,  93,  88,  40,  49, 128, 129,  58, 117,  28, 115,
        87,  92, 103, 100,  63,  35,  45,  99, 117,  45,  27,  86,  20,
        18, 133,  15,   6, 145, 104,  56,  25,  68, 144,  41,  51,  81,
        14,  67,  10, 127, 113, 123,  17,   8,  28], dtype=int64)

In [66]:
args = args_result
instance_q = test_dataset[1]
dk_list = [-1,1]
select_args = mode_para_selection(instance_q,dk_list, *args)
# get_post_Sigma_q_and_mu_q(instance_q,*select_args)
n_q = len(instance_q["input"])
[hat_D,w,mu_Gamma,C_Gamma,sigma_2] = select_args
t = 0
t_nl_t = n_q + t
Phi_t_nl_t = np.array([1, t_nl_t / 500, (t_nl_t / 500) * (t_nl_t / 500)], 
                      dtype=np.float64).reshape(1,3)
mu_q, Sigma_q = get_post_Sigma_q_and_mu_q(instance_q,*select_args)
mu_t_nl_t = Phi_t_nl_t @ mu_q
sigma_t_nl_t = Phi_t_nl_t @ Sigma_q @ Phi_t_nl_t.T
if mu_t_nl_t_0<hat_D:
    A = ((hat_D - mu_t_nl_t) / (np.sqrt(sigma_t_nl_t))).item()
else :
    A = ((-hat_D + mu_t_nl_t) / (np.sqrt(sigma_t_nl_t))).item()
Phi_t = 1 - scipy.stats.norm.cdf(A)
# get_CDF_t(instance_q,0,*select_args)


In [68]:
mu_t_nl_t,Phi_t 

(array([[-0.68126908]]), 1.0)

In [61]:
args 

[[0.56,
  array([[-0.14107795],
         [-0.07269193],
         [-0.1833172 ],
         [-0.0283842 ],
         [-0.28645507],
         [ 0.01634209]]),
  [array([[ 0.80116542],
          [-0.49698436],
          [ 6.19754031]]),
   array([[ 0.13931929, -0.03756375, -0.36207992],
          [-0.03756375,  0.14939987, -0.28534003],
          [-0.36207992, -0.28534003,  6.09838779]]),
   0.0655713138859101],
  [array([[ 0.82468967],
          [-0.42851781],
          [ 5.50735066]]),
   array([[ 0.11349203,  0.03677791, -0.56846498],
          [ 0.03677791,  0.16090321, -0.50379678],
          [-0.56846498, -0.50379678,  6.20848893]]),
   0.0646515251422192],
  [array([[ 0.58496356],
          [-0.37879269],
          [ 7.09347277]]),
   array([[ 0.11858449, -0.06840603, -0.35377243],
          [-0.06840603,  0.2823017 , -0.6051624 ],
          [-0.35377243, -0.6051624 ,  8.35371785]]),
   0.04456603529665039],
  [array([[-0.46930976],
          [ 0.39135126],
          [-7.80706822]]),
